In [1]:
#Importación de librerias útiles
import pandas
corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')

#Librerias cientificas scipy, numpy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math

#Lectura de archivos xml
from lxml import etree

#Expresiones regulares
import re

#Pyfreeling
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Para aplicar Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(stop_words='english')

In [13]:
text_list=corpus_filtro4median['text'][:]
category_list=[]
median_list=corpus_filtro4median['median'][:]
h1_list=corpus_filtro4median['1'][:]
h2_list=corpus_filtro4median['2'][:]
h3_list=corpus_filtro4median['3'][:]
h4_list=corpus_filtro4median['4'][:]
h5_list=corpus_filtro4median['5'][:]

In [14]:
pos=0
for m in median_list:
    if(m<1):
        category_list.append(0)
    else:
        category_list.append(1)
    pos+=1

In [15]:
print(len(category_list), " ", len(text_list), " ", len(median_list))

3438   3438   3438


In [16]:
print("Guardando corpus y categorias")
d = {'text' : text_list,
    'category': category_list,
     'median': median_list
    }
df = pandas.DataFrame(d, columns = ['text','category','median'])
df.to_csv('corpus_filtro4median_category.csv')
print("Tweets guardados en corpus_filtro4median_category.csv")

Guardando corpus y categorias
Tweets guardados en corpus_filtro4median_category.csv


In [2]:
vectorizer = CountVectorizer(stop_words='english')

In [17]:
corpus_filtro4median_category = pandas.read_csv("corpus_filtro4median_category.csv",encoding='utf-8')
text=corpus_filtro4median_category['text'][:]
category=corpus_filtro4median_category['category'][:]
median=corpus_filtro4median_category['median'][:]



# Separacion de datos de entrenamiento y desarrollo

In [19]:
num_dev_tweets=math.floor(len(corpus_filtro4median_category)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4median_category) - 1, size=num_dev_tweets)

print(num_dev_tweets)
print(random_samples)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4median_category['text'][:]
vec_category=corpus_filtro4median_category['category'][:]

for i in range( len(corpus_filtro4median_category)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])
        
print("Cantidad de tweets para entrenamiento", len(text_train))
print("Cantidad de tweets para desarrollo", len(text_dev))


687
[ 833 2177 3214  140 2393 2157  648 1281 3240 1231 1712 3388 3061 3032  920
 2800 3294  231  654 1809 2595 1164 1297   34 1260 2386  923 3398  411 3350
 2129  186 1651 1236 1564 2035 3149 2904 1828 2589 1667 1865 1937 1044 1459
 1607 2671 3344 2849  917  631  997 2037  743 2987  863 1708 2800 2675 1108
   27 1148 1682  273 2407 3215  180 3186 1079  761 1582  840 2877 2418  778
 3162  979 1243  270 1107  245 2028 1508 1852  917  274 2204 1089 3121 1381
 2721 2043 1764  452 1184 2773 1734 3269 3080 2251  955  640  803 1507  641
 1931  382  453 2689 2575   39 2606  982 2935  531  555 1131 1565 2284 2815
 1696 1043  238  263 2416 1595 1875  962  899 3061 2087  308 1195 2073   11
 3430 2431 1303 2735 2213   50 2591  136 2127 2798 1613 1028 2809  148 1240
 1280 1480 1571 1808 3054  879 2896  508  304  288  424 3217 1101  726  203
  542  407  568  653 3061 3026  735 3313 1760  391 1737  127  159 1339 2688
 3147  484 3414 2884 2260 1390 3240 2859  951 2413 1051 1915 1441  364 1610
 1153  5

In [20]:
print("Guardando corpus y categorias para los conjuntos de entrenamiento y desarrollo")
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset_c3.csv')
print("Tweets guardados en corpus_filtro5_trainingset_c3.csv")
d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset_c3.csv')
print("Tweets guardados en corpus_filtro5_devset_c3.csv")

Guardando corpus y categorias para los conjuntos de entrenamiento y desarrollo
Tweets guardados en corpus_filtro5_trainingset_c3.csv
Tweets guardados en corpus_filtro5_devset_c3.csv


In [21]:
corpus_filtro5_trainingset_c3 = pandas.read_csv("corpus_filtro5_trainingset_c3.csv",encoding='utf-8')
text_list=corpus_filtro5_trainingset_c3['text'][:]
category_list=corpus_filtro5_trainingset_c3['category'][:]
pos=0
for t in text_list:
    if type(t) != str:
        text_list[pos]="NaN"
    pos+=1
train_features = vectorizer.fit_transform(text_list)

In [22]:
corpus_filtro5_devset_c3 = pandas.read_csv("corpus_filtro5_devset_c3.csv",encoding='utf-8')
text_devlist=corpus_filtro5_devset_c3['text'][:]
category_devlist=corpus_filtro5_devset_c3['category'][:]
pos=0
for t in text_devlist:
    if type(t) != str:
        text_devlist[pos]="NaN"
    pos+=1
test_features = vectorizer.transform(text_devlist)

In [23]:
nb = MultinomialNB()
nb.fit(train_features , category_list)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
predictions = nb.predict(test_features)

In [25]:
from sklearn.naive_bayes import MultinomialNB
print(predictions)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 